In [30]:
import os
from datetime import datetime
from pathlib import Path
from typing import List

from dateutil.relativedelta import relativedelta
from openpyxl import Workbook
from openpyxl.chart import PieChart, Reference
from openpyxl.chart.series import DataPoint
from openpyxl.styles import Font
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.worksheet.worksheet import Worksheet  # for type hints
import pandas as pd

In [2]:
    def friendly_size(size:int) -> str:
        """Convert a size in bytes (as float) to a size with unit (as a string)"""
        size = float(size)
        unit = "B"
        # Reminder: 1 KB = 1024 B, and 1 MB = 1024 KB, ...
        for letter in "KMG":
            if size > 1024:
                size /= 1024
                unit = f"{letter}B"

        # We want to keep 2 digits after floating point
        # because it is a good balance and precision and concision
        return f"{size:0.2f} {unit}"

In [3]:
def get_file_type(path:Path):
    """Get the type of file"""
    # We might not have a suffix for files like ".gitignore"
    if path.name.startswith("."):
        return path.name.lstrip(".")

    types = {
        ".html": "HTML",
        ".htm": "HTML",
        ".ipynb": "Jupyter",
        ".xlsx": "Excel",
        ".xls": "Excel",
        ".docx": "MS Word",
        ".doc": "MS Word",
        ".txt": "Text",
        ".py": "Python",
        ".csv": "Data",
        ".json": "Data",
        ".yaml": "Data",
        ".bat": "Batch",
        ".cmd": "Batch",
        ".sh": "Batch",
    }
    return types.get(path.suffix, path.suffix.lstrip("."))

In [4]:
root = Path("e:/AnsysDev/_unpackAI/unpackai_python")

KEY_LAST_MODIF = "Last Modif Time"
KEY_SIZE_B = "Size (b)"

def iter_files(root:Path, exclude_dir:List[str]):
    """Return all info of files found in a root directory"""
    for f in root.rglob("*.*"):
        if f.is_file() and not any(d in f.parts for d in exclude_dir):
            size = f.stat().st_size
            yield {
                "Name": f.name,
                "Path": f.as_posix(),
                "Extension": f.suffix or f.name,
                "Type": get_file_type(f),
                "Size": friendly_size(size),
                KEY_SIZE_B: size,
                KEY_LAST_MODIF: datetime.fromtimestamp(f.stat().st_mtime)
            }
        
df = pd.DataFrame(iter_files(root, [".svn", ".git", ".mypy_cache"]))


df.head()

,Name,Path,Extension,Type,Size,Size (b),Last Modif Time
0,.gitignore,e:/AnsysDev/_unpackAI/unpackai_python/.gitignore,.gitignore,gitignore,12.00 B,12,2021-05-31 14:37:06.392407
1,Assignments_W02.ipynb,e:/AnsysDev/_unpackAI/unpackai_python/Assignme...,.ipynb,Jupyter,11.73 KB,12014,2021-06-07 20:22:06.025410
2,Assignments_W03.ipynb,e:/AnsysDev/_unpackAI/unpackai_python/Assignme...,.ipynb,Jupyter,12.08 KB,12368,2021-06-15 13:21:21.969640
3,Assignments_W04.ipynb,e:/AnsysDev/_unpackAI/unpackai_python/Assignme...,.ipynb,Jupyter,14.24 KB,14578,2021-06-18 20:58:42.399699
4,empty_file.txt,e:/AnsysDev/_unpackAI/unpackai_python/empty_fi...,.txt,Text,0.00 B,0,2021-06-11 20:33:30.442362


In [5]:
TABLE_LIST_FILES = "ListFiles"
FORMAT_DATE = "[$-en-US]m/d/yy h:mm AM/PM;@"

def _write_list_files(ws: Worksheet, df: pd.DataFrame):
    """Write the list of files in the given worksheet"""
    ws.title = "List of Files"

    n_row = 0
    for r in dataframe_to_rows(df, index=False, header=True):
        n_row += 1
        ws.append(r)

    # We need to adjust the column width and styles
    ws.delete_rows(2)
    width_format = {
        "Name": (30, None),
        "Path": (100, None),
        "Extension": (10, None),
        "Type": (10, None),
        "Size": (10, None),
        KEY_SIZE_B: (10, "#,##0_);(#,##0)"),
        KEY_LAST_MODIF: (20, FORMAT_DATE),
    }
    for j in range(1, len(df.columns) + 1):
        header = ws.cell(row=1, column=j).value
        width, nb_format = width_format.get(header, (None, None))
        col = get_column_letter(j)
        if width:
            ws.column_dimensions[col].width = width
        if nb_format:
            for cell in ws[col]:
                cell.number_format = nb_format

    # Add a Table with style with striped rows and banded columns
    last_col = get_column_letter(len(df.columns))
    table = Table(displayName=TABLE_LIST_FILES, ref=f"A1:{last_col}{n_row}")
    style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True)
    table.tableStyleInfo = style
    ws.add_table(table)


In [31]:
def _write_summary(ws: Worksheet, rootdir:Path, df:pd.DataFrame):
    """Create a summary of data in a given Worksheet"""

    df_ext = df.groupby("Type")[KEY_SIZE_B].agg([min, max, len, sum])
    def write_top_ten(by:str, init_row:int, col_nb:int, descr:str):
        """Write top 10 File Types with min / max / nb / sum and return last row"""
        df_top = df_ext.sort_values(by=by, ascending=False).reset_index()[:10]
        ws.append([f"Top 10 Types by {descr}"])
        ws.cell(row=init_row, column=1).style = "Headline 1"

        n_row = init_row
        for r in dataframe_to_rows(df_top, index=False, header=True):
            n_row += 1
            ws.append([None] + r)
        
        # We want to store the values as a table to have a nice style
        table = Table(displayName=f"TopType{by.capitalize()}", ref=f"B{init_row + 1}:F{n_row}")
        style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True)
        table.tableStyleInfo = style
        ws.add_table(table)

        # We want to create a Pie Chart with the top element separated
        # Note that we have an empty column so we need to shift
        pie = PieChart()
        labels = Reference(ws, min_col=2, min_row=init_row + 2, max_row=n_row)
        data = Reference(ws, min_col=col_nb + 1, min_row=init_row + 2, max_row=n_row)
        pie.add_data(data, titles_from_data=False)
        pie.set_categories(labels)
        pie.title = f"Top 10 Types by {descr}"
        slice_ = DataPoint(idx=0, explosion=20)
        pie.series[0].data_points = [slice_]
        ws.add_chart(pie, f"H{init_row}")


    ws.append(["Root Directory:", str(rootdir)])
    ws.append(["Generation Time:", datetime.now()])
    ws.cell(row=2,column=2).number_format = FORMAT_DATE
    ws["A1"].font = Font(bold=True)
    ws["A2"].font = Font(bold=True)
    ws["B1"].font = Font(italic=True)
    ws["B2"].font = Font(italic=True)
    for col in "AB":
        ws.column_dimensions[col].width = 18
    ws.append([])

    n_row = 4
    write_top_ten("sum", n_row, 5, "Total Size")
    n_row += 12 

    for _ in range(3):
        ws.append([])
        n_row += 1  
    
    write_top_ten("len", n_row, 4, "Nb of files")



In [32]:
xlsx = "from_pandas.xlsx"
wb = Workbook()
ws = wb.active
_write_list_files(ws, df)

ws = wb.create_sheet("Summary", 0)
_write_summary(ws, root, df)

wb.save(xlsx)
os.startfile(xlsx)


In [15]:
    df_ext = df.groupby("Type")[KEY_SIZE_B].agg([min, max, len, sum])
    df_ext_by_size = df_ext.sort_values(by="sum", ascending=False)[:10]
    df_ext_by_nb = df_ext.sort_values(by="len", ascending=False)[:10]
    list(df_ext_by_nb.reset_index().Type)
    df_ext

,min,max,len,sum
Type,,,,
Data,9,20592,4,20709
Excel,4864,13203,4,33515
HTML,442,1003,2,1445
Jupyter,6034,150678,19,599434
Python,252,3471,6,6566
Text,0,39212,8,39596
css,405,405,1,405
gitignore,12,12,1,12
lock,8470,8470,1,8470


# Modified Dates

* 3 days
* 2 weeks
* 6 months
* 1 year
* 3 years
* \> 3 years

In [9]:
datetime.now() - relativedelta(hours=2)

modif_times = {
    "2 hours": relativedelta(hours=2),
    "3 days": relativedelta(days=3),
    "6 months": relativedelta(months=6),
    "1 year": relativedelta(years=1),
    "3 years": relativedelta(years=3),
}

datetime.datetime(2021, 6, 20, 9, 8, 26, 713287)